## Качаем датасетик

| Критерий                             | Обоснование                                                                                                                                                  |
| ------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **Реальная практическая значимость** | Болезни растений снижают урожайность на 20‑40 %. Быстрая диагностика по фото листьев помогает агрономам и фермерам вовремя применять меры защиты.            |
| **Разнообразие и баланс классов**    | 38 меток (болезненные и здоровые листья 10 культур) → задача многоклассовой классификации, богатая на меж‑ и внутриклассовые вариации.                       |
| **Размер и доступность**             | \~54 000 RGB‑изображений 256×256 px: достаточно данных для глубоких моделей, но объём не требует распределённых систем — студенческий ПК/Colab справится.    |
| **Качество аннотаций**               | Метки проставлены специалистами Корнеллского университета; наличие как «здоровых», так и «болезненных» классов упрощает формирование отрицательных примеров. |
| **Возможность расширения**           | Позволяет тестировать аугментации (flip, color jitter, CutMix), transfer learning (ResNet, ViT, Swin) и кастомные CNN с вниманием к мелким пятнам.           |

In [1]:
!pip install kaggle --quiet

In [2]:
!mkdir -p ~/.kaggle

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset --unzip -p ./data

Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0
 95%|████████████████████████████████████▎ | 1.95G/2.04G [00:01<00:00, 1.21GB/s]
100%|██████████████████████████████████████| 2.04G/2.04G [00:01<00:00, 1.22GB/s]


## Импорты

In [1]:
import math
import random
import sys
import time
import warnings

import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models, transforms
from torchvision.transforms import RandAugment
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score

torch.backends.cudnn.benchmark = True

scaler = GradScaler()

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu"
)
DATA_DIR = "/home/data/plantvillage dataset"

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Полезные штучки

### Лоадер

In [2]:
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
IMG_SIZE = 224

def make_loaders(batch: int):
    return (
        DataLoader(
            train_ds,
            batch_size=batch,
            shuffle=True,
            num_workers=6,
            pin_memory=True,
        ),
        DataLoader(
            val_ds,
            batch_size=batch,
            shuffle=False,
            num_workers=6,
            pin_memory=True,
        ),
    )

### Аугментируйся, машина

* **Аугментации для обучения**
  Делается случайное кадрирование, горизонтальное отражение, два произвольных преобразования RandAugment, лёгкий поворот ±10°, затем перевод в тензор и нормализация. Цель — diversировать изображения, чтобы сеть не переучивалась на фиксированные ракурсы и цвета.

* **Преобразования для валидации**
  Изображение лишь масштабируется с запасом, жёстко центрируется до нужного размера и нормализуется. Никаких случайностей, чтобы метрика была стабильной.

* **Разделение 80 / 20**
  Отсек 80 % кадров идёт на обучение, 20 % — на проверку. Валидационной части подменяют «случайные» аугментации на детерминированные.

In [3]:
train_tfms = transforms.Compose(
    [
        transforms.RandomResizedCrop(
            IMG_SIZE,
            scale=(0.8, 1.0),
        ),
        transforms.RandomHorizontalFlip(),
        RandAugment(num_ops=2, magnitude=9),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(MEAN, STD),
    ]
)

val_tfms = transforms.Compose(
    [
        transforms.Resize(IMG_SIZE + 32),
        transforms.CenterCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(MEAN, STD),
    ]
)

full_ds = datasets.ImageFolder(
    root=DATA_DIR,
    transform=train_tfms,
)
num_cls = len(full_ds.classes)

train_len = int(0.8 * len(full_ds))
val_len = len(full_ds) - train_len
train_ds, val_ds = random_split(
    full_ds,
    [train_len, val_len],
)
val_ds.dataset.transform = val_tfms

print(
    f"Классов: {num_cls} | "
    f"Train: {len(train_ds)} | "
    f"Val:   {len(val_ds)}"
)

Классов: 3 | Train: 130332 | Val:   32584


### Обучалка + метрики

**Сбор статистики** — для каждого батча накапливаются:
   * сумма потерь (для усреднения в конце);
   * число верных предсказаний (для точности).

В отдельном проходе без градиентов собираются все предсказания и истинные метки, после чего считаются три показателя качества:

| Метрика               | Что показывает                                                                                                                                                    | Почему полезна                                                                                                 |
| --------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------- |
| **Accuracy**          | Простая доля совпадений «угадал / всего».                                                                                                                         | Быстрый ориентир, но может вводить в заблуждение, если классы несбалансированы.                                |
| **Weighted F1‑score** | Для каждого класса берётся F1, затем усредняется с весами, пропорциональными числу примеров класса.                                                               | Балансирует вклад частых и редких классов; важна при дисбалансе.                                               |
| **Cohen’s κ**         | Сравнивает согласие «модель‑эксперт» с учётом того, сколько совпадений ожидалось случайно. Значения: 1 — полное совпадение, 0 — как случай, <0 — хуже случайного. | Даёт более строгую оценку, показывая, насколько модель действительно «понимает» классы, а не просто угадывает. |

In [4]:
def epoch_step(
    model,
    loader,
    criterion,
    optimizer=None,
    acc_steps: int = 1,
):
    train_phase = optimizer is not None

    if train_phase:
        model.train()
    else:
        model.eval()

    total = 0
    correct = 0
    running_loss = 0.0
    grad_step = 0

    for x, y in tqdm(loader, leave=False):
        x = x.to(DEVICE)
        y = y.to(DEVICE)

        if train_phase and grad_step == 0:
            optimizer.zero_grad(set_to_none=True)

        with autocast():
            logits = model(x)
            loss = criterion(logits, y) / acc_steps

        if train_phase:
            scaler.scale(loss).backward()
            grad_step += 1

            if grad_step == acc_steps:
                scaler.step(optimizer)
                scaler.update()
                grad_step = 0

        running_loss += loss.item() * acc_steps * y.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    return running_loss / total, correct / total


@torch.no_grad()
def evaluate_metrics(model, loader):
    model.eval()
    preds = []
    labels = []

    for x, y in loader:
        logits = model(x.to(DEVICE))
        preds.append(logits.argmax(dim=1).cpu())
        labels.append(y)

    preds = torch.cat(preds)
    labels = torch.cat(labels)

    return (
        accuracy_score(labels, preds),
        f1_score(labels, preds, average="weighted"),
        cohen_kappa_score(labels, preds),
    )

## Конфиги

In [5]:
HP = {
    "res18_base": dict(batch=256, epochs=6,  lr=1e-3, acc_steps=1, aug=False, ckpt=False),
    "vit_base":   dict(batch=64,  epochs=6,  lr=3e-4, acc_steps=1, aug=False, ckpt=True),
    "res18_aug":  dict(batch=192, epochs=10, lr=3e-4, acc_steps=1, aug=True,  ckpt=False),
    "vit_aug":    dict(batch=48,  epochs=10, lr=3e-4, acc_steps=1, aug=True,  ckpt=True),
    "cnn":        dict(batch=512, epochs=15, lr=1e-3, acc_steps=1, aug=True,  ckpt=False),
    "tiny_vit":   dict(batch=256, epochs=20, lr=5e-4, acc_steps=2, aug=True,  ckpt=False),
}

## Свои и не чужие

### Обыкновенный

| Блок                  | Что делает                                        |
| :-------------------- | :------------------------------------------------ |
| **Conv 3×3 → 32…256** | 4 свёртки, каналы: 32→64→128→256                  |
| **BatchNorm + ReLU**  | Стабилизирует признаки и вводит нелинейность      |
| **MaxPool 2×2**       | Каждый раз вдвое уменьшает H×W                    |
| **AdaptiveAvgPool**   | Усредняет пространства → вектор 1×256             |
| **Linear → nc**       | Преобразует 256-мерный вектор в логиты по классам |

In [6]:
class SimpleCNN(nn.Module):
    def __init__(self, nc=num_cls):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=32,
                kernel_size=3,
                padding=1,
            ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                padding=1,
            ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                padding=1,
            ),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(
                in_channels=128,
                out_channels=256,
                kernel_size=3,
                padding=1,
            ),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
        )
        self.head = nn.Linear(256, nc)

    def forward(self, x):
        x = self.features(x)
        x = x.flatten(1)
        return self.head(x)

### Необыкновенный, но это не точно

| Компонент                        | Что делает                                                                                   |
| :------------------------------- | :------------------------------------------------------------------------------------------- |
| **Patch Embedding**              | Разбивает изображение на патчи 16×16 и проектирует их в пространство размерности 256         |
| **cls\_token + Positional Emb.** | Добавляет специальный токен «\[CLS]» и позиционные векторы для сохранения порядка патчей     |
| **Transformer Encoder × 4**      | 4 слоя самовнимания (4 головы) + MLP (×2) + Dropout → обрабатывают последовательность патчей |
| **LayerNorm**                    | Нормализует итоговый embedding «\[CLS]» перед классификацией                                 |
| **Linear Head → nc**             | Переводит 256-мерный вектор «\[CLS]» в логиты по числу классов                               |

In [7]:
class TinyViT(nn.Module):
    def __init__(
        self,
        img=IMG_SIZE,
        patch=16,
        dim=256,
        depth=4,
        heads=4,
        mlp=2,
        nc=num_cls,
    ):
        super().__init__()
        self.patch = nn.Conv2d(
            in_channels=3,
            out_channels=dim,
            kernel_size=patch,
            stride=patch,
        )
        n_patches = (img // patch) ** 2
        self.cls_token = nn.Parameter(
            torch.zeros(1, 1, dim)
        )
        self.pos_embed = nn.Parameter(
            torch.randn(1, n_patches + 1, dim) * 0.02
        )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim,
            nhead=heads,
            dim_feedforward=dim * mlp,
            batch_first=True,
            norm_first=True,
            dropout=0.1,
            activation="gelu",
        )
        self.encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=depth,
        )
        self.norm = nn.LayerNorm(dim)
        self.head = nn.Linear(dim, nc)

    def forward(self, x):
        x = self.patch(x)
        x = x.flatten(2).transpose(1, 2)
        cls = self.cls_token.expand(
            x.size(0),
            -1,
            -1,
        )
        x = torch.cat((cls, x), dim=1) + self.pos_embed
        x = self.encoder(x)
        x = self.norm(x[:, 0])
        return self.head(x)

## Великая функция глобального захвата машинами

Настройка обучения:
- Функция потерь — CrossEntropy с label smoothing, если есть аугментации.
- Оптимизатор — AdamW с весовым распадом.
- Планировщик learning rate — OneCycleLR на все эпохи.

| Модель      | Архитектура                | Аугментации         | Label smoothing | Чекпоинтинг | batch | epochs |    lr | acc\_steps |
| :---------- | :------------------------- | :------------------ | :-------------- | :---------- | ----: | -----: | ----: | ---------: |
| res18\_base | ResNet-18 (pre-ImageNet)   | базовые (crop+flip) | 0.0             | нет         |   256 |      6 | 1 e-3 |          1 |
| vit\_base   | ViT-B/16 (pre-ImageNet)    | базовые             | 0.0             | да          |    64 |      6 | 3 e-4 |          1 |
| res18\_aug  | ResNet-18                  | расширенные         | 0.1             | нет         |   192 |     10 | 3 e-4 |          1 |
| vit\_aug    | ViT-B/16                   | расширенные         | 0.1             | да          |    48 |     10 | 3 e-4 |          1 |
| cnn         | SimpleCNN (4 Conv блока)   | расширенные         | 0.1             | нет         |   512 |     15 | 1 e-3 |          1 |
| tiny\_vit   | TinyViT (patch=16, 4 слоя) | расширенные         | 0.1             | нет         |   256 |     20 | 5 e-4 |          2 |

Аугментации:
- базовые – RandomResizedCrop + RandomHorizontalFlip
- расширенные – полный train_tfms с RandAugment, RandomRotation и др.

In [8]:
def fit_model(name: str) -> dict:
    cfg = HP[name]

    global train_tfms
    if cfg["aug"]:
        train_ds.dataset.transform = train_tfms
    else:
        train_ds.dataset.transform = transforms.Compose(
            [
                transforms.RandomResizedCrop(IMG_SIZE),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(MEAN, STD),
            ]
        )

    train_dl, val_dl = make_loaders(
        batch=cfg["batch"],
    )

    if name.startswith("res18"):
        model = models.resnet18(
            weights=models.ResNet18_Weights.IMAGENET1K_V1,
        )
        model.fc = nn.Linear(
            model.fc.in_features,
            num_cls,
        )
    elif name.startswith("vit"):
        model = models.vit_b_16(
            weights=models.ViT_B_16_Weights.IMAGENET1K_V1,
        )
        model.heads.head = nn.Linear(
            model.heads.head.in_features,
            num_cls,
        )
        if cfg["ckpt"]:
            model.encoder.gradient_checkpointing = True
    elif name == "cnn":
        model = SimpleCNN()
    elif name == "tiny_vit":
        model = TinyViT()
    else:
        raise ValueError(f"Unknown model name: {name}")

    model = model.to(DEVICE)

    # Блок 3: Подготовка к обучению
    criterion = nn.CrossEntropyLoss(
        label_smoothing=0.1 if cfg["aug"] else 0.0,
    )
    optimizer = optim.AdamW(
        params=model.parameters(),
        lr=cfg["lr"],
        weight_decay=1e-4,
    )
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer=optimizer,
        max_lr=cfg["lr"],
        epochs=cfg["epochs"],
        steps_per_epoch=len(train_dl) // cfg["acc_steps"],
    )

    # Блок 4: Цикл обучения
    print(
        f"\n─── ▶️  Training {name} "
        "───────────────────────────"
    )
    for epoch in range(1, cfg["epochs"] + 1):
        epoch_step(
            model=model,
            loader=train_dl,
            criterion=criterion,
            optimizer=optimizer,
            acc_steps=cfg["acc_steps"],
        )
        scheduler.step()
        _, val_acc = epoch_step(
            model=model,
            loader=val_dl,
            criterion=criterion,
        )
        print(
            f"Epoch {epoch:02d}/"
            f"{cfg['epochs']}: val {val_acc:.4f}"
        )

    acc, f1, kappa = evaluate_metrics(
        model=model,
        loader=val_dl,
    )
    print(
        f"{name} → "
        f"Acc {acc:.4f} | "
        f"F1w {f1:.4f} | "
        f"κ {kappa:.4f}"
    )

    return {
        "Model": name,
        "Accuracy": acc,
        "Weighted F1": f1,
        "Cohen κ": kappa,
    }

## Кажется началось восстание машин

In [9]:
results = []
for m in ["res18_base", "vit_base",
          "res18_aug",  "vit_aug",
          "cnn", "tiny_vit"]:
    results.append(fit_model(m))


─── ▶️  Training res18_base ───────────────────────────


Epoch 01/6: val 0.9907


Epoch 02/6: val 0.9922


Epoch 03/6: val 0.9914


Epoch 04/6: val 0.9916


...

Epoch 17/20: val 0.9799


Epoch 18/20: val 0.9829


Epoch 19/20: val 0.9823


Epoch 20/20: val 0.9815


tiny_vit → Acc 0.9824 | F1w 0.9824 | κ 0.9735


## Результаты захвата машинами

In [10]:
df = pd.DataFrame(results)
print("\n===== Итоговые результаты =====")
print(df.to_string(index=False))


===== Итоговые результаты =====
     Model  Accuracy  Weighted F1  Cohen κ
res18_base  0.993156     0.993156 0.989734
  vit_base  0.992512     0.992511 0.988767
 res18_aug  0.991345     0.991345 0.987018
   vit_aug  0.991775     0.991775 0.987662
       cnn  0.985269     0.985265 0.977903
  tiny_vit  0.982353     0.982350 0.973529
